# Tigrigna Dictionary Updater

This notebook processes a Tigrigna corpus file and adds new words to the dictionary.

In [ ]:
import re

def load_dictionary(file_path='tigrigna_dictionary.txt'):
    """
    Loads the Tigrigna dictionary from the specified file path.
    
    Args:
        file_path (str): Path to the dictionary file
        
    Returns:
        set: A set containing Tigrigna words
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            dictionary = {line.strip() for line in file if line.strip()}
        return dictionary
    except FileNotFoundError:
        print(f"Error: Dictionary file '{file_path}' not found.")
        return set()
    except Exception as e:
        print(f"Error loading dictionary: {e}")
        return set()

# Load the current dictionary
existing_dictionary = load_dictionary()
print(f"Loaded {len(existing_dictionary)} words from existing dictionary")

In [ ]:
def tokenize_text(text):
    """
    Tokenizes Tigrigna text into words.
    
    Args:
        text (str): The input Tigrigna text
        
    Returns:
        list: A list of words
    """
    # Split text by non-Tigrigna characters and spaces
    # This regex pattern handles Tigrigna characters
    words = re.findall(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]+', text)
    
    # Remove empty strings
    words = [word.strip() for word in words if word.strip()]
    
    return words

# Test the tokenization function with a sample text
sample_text = "ሰላም ዓለም። ኣነ ኣብ ገዛ ኣለኹ።"
tokens = tokenize_text(sample_text)
print(f"Sample tokens: {tokens}")

In [ ]:
def process_corpus(corpus_file, dictionary_file):
    """
    Process a corpus file and add new words to the dictionary
    
    Args:
        corpus_file (str): Path to the corpus file
        dictionary_file (str): Path to the dictionary file
        
    Returns:
        tuple: (total_words, new_words_added, new_words_list)
    """
    # Load existing dictionary
    existing_dictionary = load_dictionary(dictionary_file)
    print(f"Loaded {len(existing_dictionary)} words from existing dictionary")
    
    # New words to add
    new_words = set()
    total_words = 0
    
    # Process corpus file
    try:
        with open(corpus_file, 'r', encoding='utf-8') as file:
            content = file.read()
            
            # Tokenize the content
            tokens = tokenize_text(content)
            total_words = len(tokens)
            
            # Find new words
            for word in tokens:
                if word not in existing_dictionary and len(word) > 1:  # Ignore single characters
                    new_words.add(word)
    
        # Add new words to dictionary file
        if new_words:
            with open(dictionary_file, 'a', encoding='utf-8') as file:
                for word in sorted(new_words):
                    file.write(f"\n{word}")
            
            print(f"Added {len(new_words)} new words to the dictionary")
        else:
            print("No new words to add to the dictionary")
            
        return total_words, len(new_words), sorted(list(new_words))
            
    except FileNotFoundError:
        print(f"Error: Corpus file '{corpus_file}' not found.")
        return 0, 0, []
    except Exception as e:
        print(f"Error processing corpus: {e}")
        return 0, 0, []

In [ ]:
# Process the corpus file
corpus_file = "tigrigna_corpus.txt"
dictionary_file = "tigrigna_dictionary.txt"

print(f"Processing corpus file: {corpus_file}")
total_words, new_words_count, new_words_list = process_corpus(corpus_file, dictionary_file)

print(f"\nSummary:")
print(f"Total words processed: {total_words}")
print(f"New words added: {new_words_count}")

# Output the updated dictionary size
updated_dict = load_dictionary(dictionary_file)
print(f"Updated dictionary now contains {len(updated_dict)} words")

In [ ]:
# Display the new words that were added
if new_words_list:
    print("New words added to the dictionary:")
    for word in new_words_list:
        print(f"- {word}")
else:
    print("No new words were added to the dictionary.")

## Process a Large Corpus

The code below can be used to process a large corpus file. It processes the file in chunks to avoid memory issues.

In [ ]:
def process_large_corpus(corpus_file, dictionary_file, chunk_size=1000000):
    """
    Process a large corpus file in chunks and add new words to the dictionary
    
    Args:
        corpus_file (str): Path to the corpus file
        dictionary_file (str): Path to the dictionary file
        chunk_size (int): Size of chunks to read at once
        
    Returns:
        tuple: (total_words, new_words_added)
    """
    # Load existing dictionary
    existing_dictionary = load_dictionary(dictionary_file)
    print(f"Loaded {len(existing_dictionary)} words from existing dictionary")
    
    # New words to add
    new_words = set()
    total_words = 0
    chunk_count = 0
    
    try:
        with open(corpus_file, 'r', encoding='utf-8') as file:
            while True:
                chunk = file.read(chunk_size)
                if not chunk:
                    break
                    
                chunk_count += 1
                print(f"Processing chunk {chunk_count}...")
                
                # Tokenize the chunk
                tokens = tokenize_text(chunk)
                total_words += len(tokens)
                
                # Find new words
                for word in tokens:
                    if word not in existing_dictionary and word not in new_words and len(word) > 1:
                        new_words.add(word)
        
        # Add new words to dictionary file
        if new_words:
            with open(dictionary_file, 'a', encoding='utf-8') as file:
                for word in sorted(new_words):
                    file.write(f"\n{word}")
            
            print(f"Added {len(new_words)} new words to the dictionary")
        else:
            print("No new words to add to the dictionary")
            
        return total_words, len(new_words)
            
    except FileNotFoundError:
        print(f"Error: Corpus file '{corpus_file}' not found.")
        return 0, 0
    except Exception as e:
        print(f"Error processing corpus: {e}")
        return 0, 0

In [ ]:
# Uncomment and run this cell to process a large corpus file
# large_corpus_file = "path_to_large_corpus.txt"  # Replace with your large corpus file path
# print(f"Processing large corpus file: {large_corpus_file}")
# total_words, new_words = process_large_corpus(large_corpus_file, dictionary_file)
# 
# print(f"\nSummary:")
# print(f"Total words processed: {total_words}")
# print(f"New words added: {new_words}")
# 
# # Output the updated dictionary size
# updated_dict = load_dictionary(dictionary_file)
# print(f"Updated dictionary now contains {len(updated_dict)} words")